In [21]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [22]:
df_genres = pd.read_parquet(r'C:\Users\EFRAIN LINARES\Desktop\Henry PI 1\PI MLOps - STEAM\PARQUET\df_dommies_clean.parquet')
df_games = pd.read_parquet(r'C:\Users\EFRAIN LINARES\Desktop\Henry PI 1\PI MLOps - STEAM\PARQUET\df_games3_clean.parquet')


In [23]:
df_genres

,id,360 Video,Accounting,Action,Adventure,Animation & Modeling,Anime,Arcade,Audio Production,Beat 'em up,...,Tutorial,Twin Stick Shooter,Utilities,VR,Video Production,Violent,War,Web Publishing,World War II,Zombies
0,761140.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,643980.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,670290.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,767400.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,773570.0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,773640.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32131,733530.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32132,610660.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32133,658870.0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
df_games

,genres,app_name,release_date,price,id,developer
0,Action,Lost Summoner Kitty,2018.0,4.99,761140,Kotoshiro
1,Indie,Ironbound,2018.0,0.00,643980,Secret Level SRL
2,Casual,Real Pool 3D - Poolians,2017.0,0.00,670290,Poolians.com
3,Action,弹炸人2222,2017.0,0.99,767400,彼岸领域
4,Action,Log Challenge,NaN,2.99,773570,None
...,...,...,...,...,...,...
32130,Casual,Colony On Mars,2018.0,1.99,773640,"Nikita ""Ghost_RUS"""
32131,Casual,LOGistICAL: South Africa,2018.0,4.99,733530,Sacada
32132,Indie,Russian Roads,2018.0,1.99,610660,Laush Dmitriy Sergeevich
32133,Casual,EXIT 2 - Directions,2017.0,4.99,658870,"xropi,stev3ns"


In [25]:
print(df_games['id'].dtype)
print(df_genres['id'].dtype)

object
float64


In [26]:
#Convertir 'id' column a str para facilitar el dataframe 
df_games['id'] = df_games['id'].astype(str)

In [28]:
df_merged = df_games.merge(df_games, on='id', how='left')

features = ['release_date'] + list(df_genres.columns[1:])  

# Normalize 'release_year'
scaler = StandardScaler()
df_merged['release_date'] = scaler.fit_transform(df_merged[['release_date']])


df_final = df_merged[['id'] + features]
df_final= df_final.merge(df_games[['id', 'app_name']], on='id', how='left')

KeyError: "None of [Index(['release_date'], dtype='object')] are in the [columns]"

In [16]:
# Sample a subset of your data, for example, 50%
df_sampled = df_final.sample(frac=0.5, random_state=42)

# Calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(df_sampled[features].fillna(0))
similarity_matrix = np.nan_to_num(similarity_matrix)

NameError: name 'df_final' is not defined

In [17]:
def recomendacion_juego_muestreado(game_id, top_n=5):
    # Obtener los IDs de los juegos muestreados
    ids_juegos_muestreados = df_sampled['id'].unique()
    
    # Verificar si el ID del juego está en los juegos muestreados
    if game_id not in ids_juegos_muestreados:
        return f"No se encontraron recomendaciones: {game_id} no está en los datos muestreados."
    
    # Obtener el índice del juego en los datos muestreados
    indice_juego = df_sampled.index[df_sampled['id'] == game_id].tolist()
    
    # Verificar si se encontró el juego en los datos muestreados
    if not indice_juego:
        return f"No se encontraron recomendaciones: {game_id} no está en los datos muestreados."
    
    indice_juego = indice_juego[0]

    # Calcular los puntajes de similitud entre juegos
    puntajes_similitud = list(enumerate(similarity_matrix[indice_juego]))
    puntajes_similitud = sorted(puntajes_similitud, key=lambda x: x[1], reverse=True)

    # Obtener los índices de los juegos similares
    indices_juegos_similares = [i for i, puntaje in puntajes_similitud[1:top_n+1]]
    
    # Obtener los nombres de los juegos similares
    nombres_juegos_similares = df_sampled['app_name'].iloc[indices_juegos_similares].tolist()

    # Mensaje de recomendación
    mensaje_recomendacion = f"Juegos recomendados basados en el ID del juego {game_id} - {df_sampled['app_name'].iloc[indice_juego]}:"
    
    return [mensaje_recomendacion] + nombres_juegos_similares

In [18]:
test_game_id = "774276"
recommendations = recomendacion_juego_muestreado(test_game_id, top_n=5)

print("Recomendaciones para ID de juego", test_game_id)
for rec in recommendations:
    print(rec)

NameError: name 'df_sampled' is not defined